# Bibliothèque à installer et import

In [1]:
#pip install llama-index
#pip install torch
#pip install sentence_transformers
#pip install InstructorEmbedding
#pip install faiss-cpu
#pip install pdfminer.six
#pip install bs4

In [18]:
import os
import glob
import torch
import json
import re
import nest_asyncio
from llama_index import VectorStoreIndex, SimpleDirectoryReader,ServiceContext,StorageContext, load_index_from_storage
from llama_index.prompts import PromptTemplate
from llama_index.llms import HuggingFaceLLM
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline
from bs4 import BeautifulSoup
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader, PDFMinerPDFasHTMLLoader
from llama_index.extractors import (
    SummaryExtractor,
    QuestionsAnsweredExtractor,
    TitleExtractor,
    KeywordExtractor,
    EntityExtractor,
    BaseExtractor,
)
from llama_index.text_splitter import TokenTextSplitter
from llama_index.ingestion import IngestionPipeline

# Paramétrage du RAG standard

### Chargement du modèle

In [2]:
# Téléchargement/ chargement du modèle mistralai/Mistral-7B-Instruct-v0.1
# 1er paramètre = nom du modèle à télécharger ou chargé si déjà téléchargé
# 2ème paramètre cache_dir permet de charger le modèle depuis un dossier donné
# 3ème paramètre device_map permet de préciser le device à utiliser (ici setup en auto pour utiliser le GPU)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",cache_dir= "/share/LLM_models/models", device_map = "auto")

# Téléchargement/ chargement du tokenizer du modèle mistralai/Mistral-7B-Instruct-v0.1
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",cache_dir= "/share/LLM_models/models", device_map = "auto")

Loading checkpoint shards: 100%|██████████| 2/2 [09:09<00:00, 274.94s/it]


### Paramétrage du LLM

In [3]:
llm = HuggingFaceLLM(
    model=model,
    tokenizer=tokenizer,
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),
    context_window=3900,
    max_new_tokens=1024,
    generate_kwargs={"temperature": 0.2, "top_k": 5, "do_sample": True},
    device_map="auto"
)


### Chargement du modèle d'embedding

In [5]:
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local:BAAI/bge-small-en-v1.5")

config.json: 100%|██████████| 743/743 [00:00<00:00, 1.75MB/s]
model.safetensors: 100%|██████████| 133M/133M [00:00<00:00, 239MB/s] 
tokenizer_config.json: 100%|██████████| 366/366 [00:00<00:00, 1.06MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 3.21MB/s]
tokenizer.json: 100%|██████████| 711k/711k [00:00<00:00, 9.77MB/s]
special_tokens_map.json: 100%|██████████| 125/125 [00:00<00:00, 409kB/s]


### Chargement des documents

In [23]:
reader_date =  SimpleDirectoryReader(input_dir = "/share/data/fiches_seismes/", required_exts=[".pdf"])
#IRSN-Seisme-Manosque-19092012.pdf & IRSN-Seisme-LaRochelle-28042016.pdf IRSN_seisme_turquie-23102011.pdf
doc_load = reader_date.load_data()

Failed to load file /share/data/fiches_seismes/IRSN-Seisme-St-Paul-Ubaye-07042014.pdf with error: Cannot read an empty file. Skipping...
Failed to load file /share/data/fiches_seismes/IRSN-Seismes-Ridgecrest-Californie-201907.pdf with error: Cannot read an empty file. Skipping...
Failed to load file /share/data/fiches_seismes/IRSN-seisme-Layon-21062019_V1.pdf with error: Cannot read an empty file. Skipping...
Failed to load file /share/data/fiches_seismes/IRSN-seisme_Okhotsk_240513.pdf with error: Cannot read an empty file. Skipping...
Failed to load file /share/data/fiches_seismes/IRSN_Fiche-seisme-PETRINJA-2020_vf.pdf with error: Cannot read an empty file. Skipping...
Failed to load file /share/data/fiches_seismes/IRSN_Fiche-seisme-Turquie-20230207.pdf with error: Cannot read an empty file. Skipping...
Failed to load file /share/data/fiches_seismes/IRSN_seisme-Chili-V3_07042010.pdf with error: Cannot read an empty file. Skipping...
Failed to load file /share/data/fiches_seismes/irsn_

### Création des vecteurs indexés

In [7]:
vector_index = VectorStoreIndex.from_documents(doc, service_context=service_context, show_progress = True)

Generating embeddings: 100%|██████████| 136/136 [00:01<00:00, 87.08it/s] 


### Sauvegarde de l'embedding

In [15]:
#/share/LLM_models/Embedding_store
vector_index.storage_context.persist(persist_dir="/share/LLM_models/Embedding_store")

### Chargement de l'embedding

In [24]:
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="/share/LLM_models/Embedding_store")

# load index
index = load_index_from_storage(storage_context,service_context=service_context)

### Paramétrage du l'outil de questions 

In [10]:
query_engine = vector_index.as_query_engine(response_mode="compact")

### Utilisation du LLM

#### Appel du query_engine pour questionner le LLM sur la base documentaire

In [11]:
reponse = query_engine.query("Quelle est la magnitude, la localisation et les conséquences régionales du séisme de Mulhouse ?")
print(reponse)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



The magnitude of the earthquake in Mulhouse is 3.9 (Mw - Géoazur). The localization of the earthquake is at 7.471° E and 47.669 °N. The consequences of the earthquake were moderate, with no damage to buildings reported. However, the earthquake was felt strongly in the region of Grand Est and caused some damage to infrastructure. The earthquake was followed by several aftershocks with magnitudes ranging from 1.0 to 3.2 on the local magnitude scale (RENASS).


#### Exemple de metadata d'une réponse du llm

In [24]:
print(reponse.metadata)

{'331f48b3-45d3-4550-b0c1-16a513ebea43': {'page_label': '1', 'file_name': '/share/data/fiches_seismes/IRSN_Fiche-Seisme-Ouest-France-16-juin-2023_V1-17062023.pdf'}, '1e99008f-119d-422d-b00c-1c4504be6211': {'page_label': '2', 'file_name': '/share/data/fiches_seismes/IRSN_Fiche-Seisme-Ouest-France-16-juin-2023_V1-17062023.pdf'}}


# Sauvegarde des réponses dans un csv

#### Dictionnaire de question pour chaque pdf fonctionnel

In [32]:
mon_dict = {
    'IRSN Fiche seisme MULHOUSE 13092022.pdf': ['Quelle est la magnitude, la localisation et les conséquences régionales du séisme de Mulhouse ?', "Comment le séisme de Mulhouse est-il lié à la tectonique du Fossé Rhénan, et quelles sont les prévisions quant à l'activité sismique future dans la région ?"],
    'IRSN_Fiche-Seisme-Maroc.pdf': ['Quelles sont les caractéristiques principales du séisme au Maroc du 8 septembre 2023, telles que sa magnitude, sa localisation, et quelles ont été ses conséquences régionales ?', "Comment s'inscrit le séisme du 8 septembre 2023 dans le contexte sismotectonique du Maroc, et quel impact a-t-il eu sur les installations nucléaires dans la région ?"],
    'IRSN_Fiche-Seisme-Ouest-France-16-juin-2023_V1-17062023.pdf': ["Quelles sont les caractéristiques principales du séisme dans l'ouest de la France du 16 juin 2023, notamment sa localisation, sa magnitude, et quels sont les impacts rapportés dans la région épicentrale ?", "Comment se situe le séisme du 16 juin 2023 par rapport à l'activité sismique historique de la région, et quel a été l'impact sur les installations nucléaires à proximité, telles que la centrale nucléaire de Civaux ?"],
    'irsn_seisme_argeles_gazost_112006.pdf': ["Pouvez-vous fournir des informations sur le séisme d'Argelès-Gazost du 17 novembre 2006, notamment sa magnitude, son épicentre, et ses impacts locaux ?", "Quel a été l'impact du séisme d'Argelès-Gazost sur les installations nucléaires de la région ?"],
    'irsn_seisme_chinon_11_2006.pdf': ['Pouvez-vous fournir des informations sur le séisme de Chinon du 5 novembre 2006, notamment sa magnitude, son épicentre, et ses impacts locaux ?', "Quel a été l'impact du séisme de Chinon sur les installations nucléaires de la région ?"],
    'IRSN_seisme_Christchurch-NZ_23022011.pdf': ['Quels sont les impacts du séisme de Christchurch du 21 février 2011 par rapport à celui de septembre 2010 en Nouvelle-Zélande?', 'Quelle est la situation des installations nucléaires en Nouvelle-Zélande et quelles sont les caractéristiques de la faille de GreenDale?'],
    'IRSN_seisme_france-26022012_V2.pdf': ['Quels sont les détails concernant le séisme à Saint-Paul-sur-Ubaye du 26 février 2012?', "Comment la sismicité historique et actuelle dans la vallée de l'Ubaye est-elle liée au séisme du 26 février 2012?"],
    'IRSN_seisme_Italie-20052012.pdf': ["Quel est le contexte sismotectonique de la région touchée par le séisme de l'Emilia-Romagna?", "Quelle est l'estimation de l'aléa sismique et du mouvement sismique en basse plaine du Pô?"],
    'IRSN_Seisme_Lorca-Espagne_18052011.pdf': ['Quels sont les détails concernant le séisme de Lorca en Espagne du 11 mai 2011?', 'Quel est le contexte sismique de la France autour du 11 mai 2011?'],
    'irsn_seisme_martinique_122007.pdf': ['Quand a eu lieu le séisme de Martinique et quelle était sa magnitude?', "Où était situé l'épicentre du séisme de Martinique?"],
    'irsn_seisme_niigata_japon_092007.pdf': ['Comment le Japon se situe-t-il du point de vue sismotectonique par rapport à la convergence des plaques tectoniques?', 'Quelles étaient les conséquences humaines du séisme de Chu-Etsu-Oki?'],
    'IRSN_seisme_sumatra-11042012.pdf': ['Quand ont eu lieu les séismes de Sumatra en Indonésie, et quelle était leur magnitude?', "Quelle était la distance entre la centrale nucléaire la plus proche (Kalpakkam) et l'épicentre du séisme de Sumatra, et quels étaient les effets attendus sur la centrale?"],
    'IRSN_seisme_turquie-23102011.pdf': ['Quels ont été les effets du séisme de Van en termes de dégâts matériels et humains?', 'Où était située la centrale nucléaire la plus proche du séisme de Van, et quelles étaient les conséquences sur cette centrale?'],
    'IRSN_seisme_virginie-US-23082011.pdf': ["Dans quelle région s'est produit le séisme de Virginie, et qu'est-ce que la « Central Virginia Seismic Zone » ?", " Quel est le contexte sismotectonique de l'Est des États-Unis, et pourquoi la sismicité n'y est-elle pas négligeable?"],
    'irsn_Seisme-Afghanistan_032002.pdf': ['Quand a eu lieu le séisme en Afghanistan, et quelle était sa magnitude?', 'Quelles sont les différentes estimations de la magnitude du séisme en Afghanistan?'],
    'irsn_seisme-algerie_052003.pdf': ["Quel est le contexte tectonique de l'Algérie et comment cela contribue-t-il à la sismicité de la région?", 'Quels sont les effets de site probables dans les localités de Rouïba et Boumerdès?'],
    'irsn_Seisme-Allemagne_122004.pdf': ['Quelle est la proximité du séisme de Fribourg par rapport à la centrale nucléaire de Fessenheim?', "Quelle est la distance entre l'épicentre du séisme de Fribourg et les centrales nucléaires suisses de Leipstadt et Beznau?"],
    'irsn_Seisme-Besancon_022004.pdf': ['Quand a eu lieu le séisme de Besançon et quelle était sa magnitude?'],
    'IRSN_seisme-haiti_130110.pdf': ["Où était situé l'épicentre du séisme d'Haïti, et quels ont été les effets notables?", "Quel est le contexte sismotectonique de la région où s'est produit le séisme d'Haïti?"],
    'irsn_Seisme-Inde_012001.pdf': ['Quels ont été les effets notables du séisme de Gurajat, et quel était le mécanisme de la faille en jeu?', ' Pourquoi le séisme de Gurajat a-t-il été considéré comme une surprise relative?'],
    'irsn_Seisme-Japon_052003.pdf': ["Quand et où s'est produit le séisme de Miyagi au Japon, et quelle était sa magnitude?", 'Quels sont les impacts du séisme de Miyagi en termes de dégâts matériels, de victimes et de répliques?'],
    'IRSN_Seisme-Japon_Replique-7-avril_V1-08042011.pdf': ["Comment la centrale nucléaire d'Onagawa a-t-elle été affectée par cette réplique en termes d'accélérations du sol?", 'Quelles sont les actions entreprises par les autorités japonaises pour évaluer les effets de cette réplique?'],
    'IRSN_seisme-Japon_V2-22042011.pdf': ["Quand et où s'est produit le séisme de Tohoku au Japon du 11 mars 2011, et quelle était sa magnitude?", "Quelle était la localisation de l'épicentre et quels sont les détails sur la magnitude du séisme de Tohoku au Japon du 11 mars 2011?"],
    'IRSN_seisme-Manosque_V1-12072010.pdf': ['Quelle est la magnitude du séisme survenu à Manosque le 8 juillet 2010, et quelle est la version du document que vous consultez ?', 'Comment la population a-t-elle réagi à la secousse sismique, et quelles sont les zones les plus fortement touchées pour le séisme du 8 juillet 2010 ? ?'],
    'irsn_Seisme-Mexique_012003.pdf': ["Quels sont les dégâts rapportés suite au séisme, et quelle est la particularité de la ville de Mexico en termes d'effet sismique ?", 'Quelle est la situation des installations nucléaires au Mexique, et comment le séisme du 21 janvier 2003 a-t-il affecté la centrale nucléaire de Laguna Verde '],
    'IRSN-Seisme-Amatrice-Italie-240816_V1.pdf': ["Quelle est la magnitude du séisme d'Amatrice qui s'est produit le 24 août 2016, et où était situé son épicentre ?", 'Quels ont été les effets du séisme  du 24 août 2016 en termes de victimes, de blessés et de dégâts matériels ?'],
    'IRSN-Seisme-Charente-Maritime-France-20190320_v1.1.pdf': ['Quels ont été les effets du séisme de Montendre en termes de ressenti par la population et et de dégâts matériels ?', 'Comment le séisme de Montendre de 2019 se compare-t-il aux plus forts séismes historiques de la région ?'],
    'IRSN-Seisme-Civaux-09092013.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Lhommaizé le 9 septembre 2013, et où était situé son épicentre ?", 'Quels ont été les effets du séisme de Lhommaizé en termes de ressenti par la population et de dégâts matériels ?'],
    'IRSN-Seisme-Kent-22052015.pdf': ["Quels ont été les impacts du séisme dans le Kent le 22 mai 2015 en termes de secousses ressenties, d'intensités et de mécanisme sismique associé ?", 'Quelle est la sismicité historique dans le détroit du pas de Calais, et quelles sont les recherches menées pour mieux comprendre le risque sismique dans cette région ?'],
    'IRSN-Seisme-Elazig-Turquie-24012020.pdf': ["Quelle est la géodynamique de la région et la sismicité régionale autour d'Elazig ?", 'Quelles sont les conséquences sur les installations nucléaires à proximité, en particulier la centrale nucléaire de Metsamor en Arménie, et quels sont les projets de centrales nucléaires en Turquie ?'],
    'IRSN-Seisme-Fukushima-13022021.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Fukushima le 13 février 2021, et où était situé son épicentre ?", 'Quels ont été les impacts du séisme de Fukushima en termes de victimes, de dommages matériels et de secousses ressenties dans la région ?'],
    'IRSN-Seisme-LaRochelle-28042016.pdf': ["Quelle est la magnitude du séisme qui s'est produit à La Rochelle le 28 avril 2016, et où était situé son épicentre ?", "Quelle est la sismicité historique et actuelle au nord de l'Aquitaine, et comment le séisme de 2016 s'inscrit-il dans ce contexte ?"],
    'IRSN-Seisme-Manosque-19092012.pdf': ["Quels ont été les faits marquants du séisme à Villeneuve le 19 septembre 2012, en termes de répliques, de secousses ressenties, et d'historique sismique de la région ?", "Quelle est la géométrie de la Faille de la Moyenne Durance, et comment le séisme de 2012 s'inscrit-il dans l'activité de cette faille ?"],
    'IRSN-Seisme-Napa-24082014.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Napa le 24 août 2014, et quelles sont les principales caractéristiques de ce séisme ?", 'Quelle est la localisation du séisme de Napa par rapport aux failles tectoniques de la région, et quelles sont les caractéristiques tectoniques de cette zone ?'],
    'IRSN-Seisme-Norcia-Italie-301016_V1.pdf': ["Quelle est la magnitude du séisme qui s'est produit en Italie centrale le 30 octobre 2016, et quelles sont les principales caractéristiques de ce séisme ?", "Quels sont les enseignements tirés de la séquence sismique en Italie centrale, et en quoi cette expérience peut-elle être importante pour la compréhension des risques sismiques dans d'autres régions, y compris en France ?"],
    'IRSN-Seisme-Vendee-France-20180212.pdf': ['Quand et où a eu lieu le séisme de Saint-Hilaire-de-Voust en 2018, quelle était sa magnitude, et quels sont les dégâts signalés?', "Quel a été l'impact du séisme de Saint-Hilaire-de-Voust de 2018 sur les installations nucléaires, et quelles sont les installations nucléaires les plus proches de l'épicentre?"]}

#### Script pour obtenir tous les résultats

In [33]:
reponses_dict = {}  
for cle, questions in mon_dict.items():
    print(f'Clé: {cle}')
    
    # Liste pour stocker les réponses aux questions
    reponses = []
    # Itérer sur chaque question pour la clé
    for i, question in enumerate(questions, start=1):
        query = question
        reponses.append(query_engine.query(query))  
    # Stocker les réponses dans le dictionnaire de réponses
    reponses_dict[cle] = reponses
# Afficher le dictionnaire de réponses
print('\nDictionnaire de réponses:')
print(reponses_dict)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN Fiche seisme MULHOUSE 13092022.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN_Fiche-Seisme-Maroc.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Clé: IRSN-Seisme-Norcia-Italie-301016_V1.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN-Seisme-Vendee-France-20180212.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Dictionnaire de réponses:
{'IRSN Fiche seisme MULHOUSE 13092022.pdf': [Response(response='\nThe magnitude of the earthquake in Mulhouse is 3.9 (Mw - Géoazur). The earthquake occurred at 15h58 TU (17h58 local time) on September 1, 2022, and was located at 7.471° E and 47.669° N. The earthquake was felt throughout the region of Grand Est and caused damage to buildings. The maximum intensity of the earthquake was IV on the RENASS scale, which corresponds to a perception that is moderately felt and did not cause any damage to buildings.', source_nodes=[NodeWithScore(node=TextNode(id_='e9d0f049-88cf-4366-85d8-91fbfb13c7fe', embedding=None, metadata={'page_label': '1', 'file_name': '/share/data/fiches_seismes/IRSN Fiche seisme MULHOUSE 13092022.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e7c34ba8-7725-4848-8b49-24e10053d92a', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', '

#### Formattage de l'output pour garder seulement les réponses + les sources

In [136]:
dict_output = {}

for cle in reponses_dict:
    if cle == 'irsn_Seisme-Besancon_022004.pdf':
        continue

    # Initialiser une liste pour stocker les réponses
    reponses_list = []

    # Itérer sur chaque question pour la clé
    for i in range(2):
        reponses_list.append({
            f'Réponse {i}': [reponses_dict[cle][i].response, reponses_dict[cle][i].metadata[list(reponses_dict[cle][i].metadata.keys())[0]]['file_name'] + ' & ' + reponses_dict[cle][i].metadata[list(reponses_dict[cle][i].metadata.keys())[1]]['file_name']]
        })

    # Ajouter la liste des réponses au dictionnaire de sortie
    dict_output[cle] = reponses_list

# Afficher le résultat pour une clé spécifique
print(dict_output['IRSN Fiche seisme MULHOUSE 13092022.pdf'])


[{'Réponse 0': ['\nThe magnitude of the earthquake in Mulhouse is 3.9 (Mw - Géoazur). The earthquake occurred at 15h58 TU (17h58 local time) on September 1, 2022, and was located at 7.471° E and 47.669° N. The earthquake was felt throughout the region of Grand Est and caused damage to buildings. The maximum intensity of the earthquake was IV on the RENASS scale, which corresponds to a perception that is moderately felt and did not cause any damage to buildings.', '/share/data/fiches_seismes/IRSN Fiche seisme MULHOUSE 13092022.pdf & /share/data/fiches_seismes/IRSN_seisme-Manosque_V1-12072010.pdf']}, {'Réponse 1': ['\nThe earthquake in Mulhouse is linked to the tectonics of the Fossé Rhénan, which is a rift system that has been forming for approximately 40 million years. The Fossé Rhénan is characterized by a system of faults oriented primarily in the direction of NNE-SSO, with several faults presenting markers of recent activity (quaternary). The mechanism of the earthquake in Mulhouse 

#### Sauvegarde de l'output formatté dans un csv

In [138]:
import csv
output_csv_path = 'output_llma.csv'
# Fonction pour écrire les données dans le fichier CSV
def write_to_csv(data, csv_path):
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        # Définir les colonnes du CSV
        fieldnames = ['Nom du document', 'Réponse 0', 'Source 0', 'Réponse 1', 'Source 1']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')  # Ajout du délimiteur

        # Écrire l'en-tête
        writer.writeheader()

        # Écrire les données
        for document, responses in data.items():
            row_data = {'Nom du document': document}
            for i, response_info in enumerate(responses):
                response_key = f'Réponse {i}'
                source_key = f'Source {i}'
                
                # Enlever le premier caractère (retour chariot) du string réponse
                cleaned_response = response_info[response_key][0][1:] if response_info[response_key][0] else ''
                
                row_data[response_key] = cleaned_response
                row_data[source_key] = str(response_info[response_key][1]) if response_info[response_key][1] else ''
            writer.writerow(row_data)

# Appeler la fonction avec vos données
write_to_csv(dict_output, output_csv_path)

# Afficher un message de confirmation
print(f"Les données ont été écrites dans le fichier CSV : {output_csv_path}")

Les données ont été écrites dans le fichier CSV : output_llma.csv


# Extraction de metadonnées 

### import et fonction d'extraction de metadata

In [15]:
text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=512, chunk_overlap=128
)

In [16]:
nest_asyncio.apply()
extractors = [
    TitleExtractor(nodes=5, llm=llm),
    QuestionsAnsweredExtractor(questions=3, llm=llm),
    # EntityExtractor(prediction_threshold=0.5),
    # SummaryExtractor(summaries=["prev", "self"], llm=llm),
    # KeywordExtractor(keywords=10, llm=llm),
    # CustomExtractor()
]

transformations = [text_splitter] + extractors

### Création des metadatas supplémentaires

In [38]:
pipeline = IngestionPipeline(transformations=transformations)
doc_nodes = pipeline.run(documents=doc)

  0%|          | 0/2 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 2/2 [00:03<00:00,  1.82s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 0/3 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 3/3 [00:05<00:00,  1.68s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 1/1 [00:01<00:00,  1.86s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 0/3 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100

In [40]:
print(len(doc_nodes))

267


### Création de l'index

In [41]:
#query_engine = index.as_query_engine(response_mode="compact")
vector_index_new_embedding = VectorStoreIndex(doc_nodes, service_context=service_context, show_progress = True)

Generating embeddings: 100%|██████████| 267/267 [00:01<00:00, 193.88it/s]


### Stockage de l'index

In [44]:
vector_index_new_embedding.storage_context.persist(persist_dir="/share/LLM_models/Embedding_store_v2")

### Chargement de l'embeddings

In [10]:
storage_context = StorageContext.from_defaults(persist_dir="/share/LLM_models/Embedding_store_v2")

### Load de l'embeddings

In [ ]:
# load index
index = load_index_from_storage(storage_context,service_context=service_context)

### Création du l'outil de question + question

In [11]:
query_engine = index.as_query_engine(response_mode="compact")

In [12]:
reponse = query_engine.query("Quels sont les détails concernant le séisme de Lorca en Espagne du 11 mai 2011?")
print(reponse)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


IRSN_Seisme_Lorca-Espagne_18052011.pdf
IRSN_Seisme_Lorca-Espagne_18052011.pdf

The Lorca Earthquake occurred on May 11, 2011, in Lorca, Spain. The earthquake was of magnitude 5.1 on the EMS98 scale and occurred at a shallow depth of 2 km. The epicenter of the earthquake was located at 37.68° S and 1.68° E (source EMSC). The earthquake caused significant damage to buildings in the medieval cities of Lorca and Totana, with an estimated 9,000 people being evacuated from Lorca, which has a population of 92,700. The preliminary intensity of the earthquake in the epicentral zone was VII on the EMS98 scale. The earthquake was preceded by a first earthquake of magnitude 4.4 at 17h05 (15h05 GMT).


## Sauvegarde csv avec extracteur

### Dictionnaire question

In [344]:
mon_dict = {
    'IRSN Fiche seisme MULHOUSE 13092022.pdf': ['Quelle est la magnitude, la localisation et les conséquences régionales du séisme de Mulhouse ?', "Comment le séisme de Mulhouse est-il lié à la tectonique du Fossé Rhénan, et quelles sont les prévisions quant à l'activité sismique future dans la région ?"],
    'IRSN_Fiche-Seisme-Maroc.pdf': ['Quelles sont les caractéristiques principales du séisme au Maroc du 8 septembre 2023, telles que sa magnitude, sa localisation, et quelles ont été ses conséquences régionales ?', "Comment s'inscrit le séisme du 8 septembre 2023 dans le contexte sismotectonique du Maroc, et quel impact a-t-il eu sur les installations nucléaires dans la région ?"],
    'IRSN_Fiche-Seisme-Ouest-France-16-juin-2023_V1-17062023.pdf': ["Quelles sont les caractéristiques principales du séisme dans l'ouest de la France du 16 juin 2023, notamment sa localisation, sa magnitude, et quels sont les impacts rapportés dans la région épicentrale ?", "Comment se situe le séisme du 16 juin 2023 par rapport à l'activité sismique historique de la région, et quel a été l'impact sur les installations nucléaires à proximité, telles que la centrale nucléaire de Civaux ?"],
    'irsn_seisme_argeles_gazost_112006.pdf': ["Pouvez-vous fournir des informations sur le séisme d'Argelès-Gazost du 17 novembre 2006, notamment sa magnitude, son épicentre, et ses impacts locaux ?", "Quel a été l'impact du séisme d'Argelès-Gazost sur les installations nucléaires de la région ?"],
    'irsn_seisme_chinon_11_2006.pdf': ['Pouvez-vous fournir des informations sur le séisme de Chinon du 5 novembre 2006, notamment sa magnitude, son épicentre, et ses impacts locaux ?', "Quel a été l'impact du séisme de Chinon sur les installations nucléaires de la région ?"],
    'IRSN_seisme_Christchurch-NZ_23022011.pdf': ['Quels sont les impacts du séisme de Christchurch du 21 février 2011 par rapport à celui de septembre 2010 en Nouvelle-Zélande?', 'Quelle est la situation des installations nucléaires en Nouvelle-Zélande et quelles sont les caractéristiques de la faille de GreenDale?'],
    'IRSN_seisme_france-26022012_V2.pdf': ['Quels sont les détails concernant le séisme à Saint-Paul-sur-Ubaye du 26 février 2012?', "Comment la sismicité historique et actuelle dans la vallée de l'Ubaye est-elle liée au séisme du 26 février 2012?"],
    'IRSN_seisme_Italie-20052012.pdf': ["Quel est le contexte sismotectonique de la région touchée par le séisme de l'Emilia-Romagna?", "Quelle est l'estimation de l'aléa sismique et du mouvement sismique en basse plaine du Pô?"],
    'IRSN_Seisme_Lorca-Espagne_18052011.pdf': ['Quels sont les détails concernant le séisme de Lorca en Espagne du 11 mai 2011?', 'Quel est le contexte sismique de la France autour du 11 mai 2011?'],
    'irsn_seisme_martinique_122007.pdf': ['Quand a eu lieu le séisme de Martinique et quelle était sa magnitude?', "Où était situé l'épicentre du séisme de Martinique?"],
    'irsn_seisme_niigata_japon_092007.pdf': ['Comment le Japon se situe-t-il du point de vue sismotectonique par rapport à la convergence des plaques tectoniques?', 'Quelles étaient les conséquences humaines du séisme de Chu-Etsu-Oki?'],
    'IRSN_seisme_sumatra-11042012.pdf': ['Quand ont eu lieu les séismes de Sumatra en Indonésie, et quelle était leur magnitude?', "Quelle était la distance entre la centrale nucléaire la plus proche (Kalpakkam) et l'épicentre du séisme de Sumatra, et quels étaient les effets attendus sur la centrale?"],
    'IRSN_seisme_turquie-23102011.pdf': ['Quels ont été les effets du séisme de Van en termes de dégâts matériels et humains?', 'Où était située la centrale nucléaire la plus proche du séisme de Van, et quelles étaient les conséquences sur cette centrale?'],
    'IRSN_seisme_virginie-US-23082011.pdf': ["Dans quelle région s'est produit le séisme de Virginie, et qu'est-ce que la « Central Virginia Seismic Zone » ?", " Quel est le contexte sismotectonique de l'Est des États-Unis, et pourquoi la sismicité n'y est-elle pas négligeable?"],
    'irsn_Seisme-Afghanistan_032002.pdf': ['Quand a eu lieu le séisme en Afghanistan, et quelle était sa magnitude?', 'Quelles sont les différentes estimations de la magnitude du séisme en Afghanistan?'],
    'irsn_seisme-algerie_052003.pdf': ["Quel est le contexte tectonique de l'Algérie et comment cela contribue-t-il à la sismicité de la région?", 'Quels sont les effets de site probables dans les localités de Rouïba et Boumerdès?'],
    'irsn_Seisme-Allemagne_122004.pdf': ['Quelle est la proximité du séisme de Fribourg par rapport à la centrale nucléaire de Fessenheim?', "Quelle est la distance entre l'épicentre du séisme de Fribourg et les centrales nucléaires suisses de Leipstadt et Beznau?"],
    'irsn_Seisme-Besancon_022004.pdf': ['Quand a eu lieu le séisme de Besançon et quelle était sa magnitude?'],
    'IRSN_seisme-haiti_130110.pdf': ["Où était situé l'épicentre du séisme d'Haïti, et quels ont été les effets notables?", "Quel est le contexte sismotectonique de la région où s'est produit le séisme d'Haïti?"],
    'irsn_Seisme-Inde_012001.pdf': ['Quels ont été les effets notables du séisme de Gurajat, et quel était le mécanisme de la faille en jeu?', ' Pourquoi le séisme de Gurajat a-t-il été considéré comme une surprise relative?'],
    'irsn_Seisme-Japon_052003.pdf': ["Quand et où s'est produit le séisme de Miyagi au Japon, et quelle était sa magnitude?", 'Quels sont les impacts du séisme de Miyagi en termes de dégâts matériels, de victimes et de répliques?'],
    'IRSN_Seisme-Japon_Replique-7-avril_V1-08042011.pdf': ["Comment la centrale nucléaire d'Onagawa a-t-elle été affectée par cette réplique en termes d'accélérations du sol?", 'Quelles sont les actions entreprises par les autorités japonaises pour évaluer les effets de cette réplique?'],
    'IRSN_seisme-Japon_V2-22042011.pdf': ["Quand et où s'est produit le séisme de Tohoku au Japon du 11 mars 2011, et quelle était sa magnitude?", "Quelle était la localisation de l'épicentre et quels sont les détails sur la magnitude du séisme de Tohoku au Japon du 11 mars 2011?"],
    'IRSN_seisme-Manosque_V1-12072010.pdf': ['Quelle est la magnitude du séisme survenu à Manosque le 8 juillet 2010, et quelle est la version du document que vous consultez ?', 'Comment la population a-t-elle réagi à la secousse sismique, et quelles sont les zones les plus fortement touchées pour le séisme du 8 juillet 2010 ? ?'],
    'irsn_Seisme-Mexique_012003.pdf': ["Quels sont les dégâts rapportés suite au séisme, et quelle est la particularité de la ville de Mexico en termes d'effet sismique ?", 'Quelle est la situation des installations nucléaires au Mexique, et comment le séisme du 21 janvier 2003 a-t-il affecté la centrale nucléaire de Laguna Verde '],
    'IRSN-Seisme-Amatrice-Italie-240816_V1.pdf': ["Quelle est la magnitude du séisme d'Amatrice qui s'est produit le 24 août 2016, et où était situé son épicentre ?", 'Quels ont été les effets du séisme  du 24 août 2016 en termes de victimes, de blessés et de dégâts matériels ?'],
    'IRSN-Seisme-Charente-Maritime-France-20190320_v1.1.pdf': ['Quels ont été les effets du séisme de Montendre en termes de ressenti par la population et et de dégâts matériels ?', 'Comment le séisme de Montendre de 2019 se compare-t-il aux plus forts séismes historiques de la région ?'],
    'IRSN-Seisme-Civaux-09092013.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Lhommaizé le 9 septembre 2013, et où était situé son épicentre ?", 'Quels ont été les effets du séisme de Lhommaizé en termes de ressenti par la population et de dégâts matériels ?'],
    'IRSN-Seisme-Kent-22052015.pdf': ["Quels ont été les impacts du séisme dans le Kent le 22 mai 2015 en termes de secousses ressenties, d'intensités et de mécanisme sismique associé ?", 'Quelle est la sismicité historique dans le détroit du pas de Calais, et quelles sont les recherches menées pour mieux comprendre le risque sismique dans cette région ?'],
    'IRSN-Seisme-Elazig-Turquie-24012020.pdf': ["Quelle est la géodynamique de la région et la sismicité régionale autour d'Elazig ?", 'Quelles sont les conséquences sur les installations nucléaires à proximité, en particulier la centrale nucléaire de Metsamor en Arménie, et quels sont les projets de centrales nucléaires en Turquie ?'],
    'IRSN-Seisme-Fukushima-13022021.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Fukushima le 13 février 2021, et où était situé son épicentre ?", 'Quels ont été les impacts du séisme de Fukushima en termes de victimes, de dommages matériels et de secousses ressenties dans la région ?'],
    'IRSN-Seisme-LaRochelle-28042016.pdf': ["Quelle est la magnitude du séisme qui s'est produit à La Rochelle le 28 avril 2016, et où était situé son épicentre ?", "Quelle est la sismicité historique et actuelle au nord de l'Aquitaine, et comment le séisme de 2016 s'inscrit-il dans ce contexte ?"],
    'IRSN-Seisme-Manosque-19092012.pdf': ["Quels ont été les faits marquants du séisme à Villeneuve le 19 septembre 2012, en termes de répliques, de secousses ressenties, et d'historique sismique de la région ?", "Quelle est la géométrie de la Faille de la Moyenne Durance, et comment le séisme de 2012 s'inscrit-il dans l'activité de cette faille ?"],
    'IRSN-Seisme-Napa-24082014.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Napa le 24 août 2014, et quelles sont les principales caractéristiques de ce séisme ?", 'Quelle est la localisation du séisme de Napa par rapport aux failles tectoniques de la région, et quelles sont les caractéristiques tectoniques de cette zone ?'],
    'IRSN-Seisme-Norcia-Italie-301016_V1.pdf': ["Quelle est la magnitude du séisme qui s'est produit en Italie centrale le 30 octobre 2016, et quelles sont les principales caractéristiques de ce séisme ?", "Quels sont les enseignements tirés de la séquence sismique en Italie centrale, et en quoi cette expérience peut-elle être importante pour la compréhension des risques sismiques dans d'autres régions, y compris en France ?"],
    'IRSN-Seisme-Vendee-France-20180212.pdf': ['Quand et où a eu lieu le séisme de Saint-Hilaire-de-Voust en 2018, quelle était sa magnitude, et quels sont les dégâts signalés?', "Quel a été l'impact du séisme de Saint-Hilaire-de-Voust de 2018 sur les installations nucléaires, et quelles sont les installations nucléaires les plus proches de l'épicentre?"]}

### Script 

In [50]:
reponses_dict = {}  
for cle, questions in mon_dict.items():
    print(f'Clé: {cle}')
    
    # Liste pour stocker les réponses aux questions
    reponses = []
    # Itérer sur chaque question pour la clé
    for i, question in enumerate(questions, start=1):
        query = question
        reponses.append(query_engine.query(query))  
    # Stocker les réponses dans le dictionnaire de réponses
    reponses_dict[cle] = reponses


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN Fiche seisme MULHOUSE 13092022.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN_Fiche-Seisme-Maroc.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN-Seisme-Norcia-Italie-301016_V1.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN-Seisme-Vendee-France-20180212.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [53]:
dict_output = {}

for cle in reponses_dict:
    if cle == 'irsn_Seisme-Besancon_022004.pdf':
        continue

    # Initialiser une liste pour stocker les réponses
    reponses_list = []

    # Itérer sur chaque question pour la clé
    for i in range(2):
        reponses_list.append({
            f'Réponse {i}': [reponses_dict[cle][i].response, reponses_dict[cle][i].metadata[list(reponses_dict[cle][i].metadata.keys())[0]]['file_name'] + ' & ' + reponses_dict[cle][i].metadata[list(reponses_dict[cle][i].metadata.keys())[1]]['file_name']]
        })

    # Ajouter la liste des réponses au dictionnaire de sortie
    dict_output[cle] = reponses_list

# Afficher le résultat pour une clé spécifique
print(dict_output['IRSN Fiche seisme MULHOUSE 13092022.pdf'])


[{'Réponse 0': ['\nThe magnitude of the earthquake that occurred on September 10, 2022 near Mulhouse and Bâle is 3.9 (Mw - Géoazur). The localization of the earthquake is 7.471° E. The consequences of the earthquake were felt throughout the region of Grand Est, with a maximum intensity of IV on the local scale, corresponding to a perception that was moderately felt by humans and caused no damage to buildings.', 'IRSN Fiche seisme MULHOUSE 13092022.pdf & IRSN-Seisme-Manosque-19092012.pdf']}, {'Réponse 1': ["\nThe 2022 Basel earthquake is related to the tectonics of the Fossé Rhénan, which is a rift system that has been forming for approximately 40 million years. The Fossé Rhénan is characterized by a system of faults oriented primarily in the NNE-SSO direction, with several faults presenting markers of recent activity (quaternary). The mechanism of the earthquake's origin determined for the 2022 Basel earthquake indicates a pure decrochant earthquake in agreement with the regional conte

In [54]:
import csv
output_csv_path = 'output_llama_extract.csv'
# Fonction pour écrire les données dans le fichier CSV
def write_to_csv(data, csv_path):
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        # Définir les colonnes du CSV
        fieldnames = ['Nom du document', 'Réponse 0', 'Source 0', 'Réponse 1', 'Source 1']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')  # Ajout du délimiteur

        # Écrire l'en-tête
        writer.writeheader()

        # Écrire les données
        for document, responses in data.items():
            row_data = {'Nom du document': document}
            for i, response_info in enumerate(responses):
                response_key = f'Réponse {i}'
                source_key = f'Source {i}'
                
                # Enlever le premier caractère (retour chariot) du string réponse
                cleaned_response = response_info[response_key][0][1:] if response_info[response_key][0] else ''
                
                row_data[response_key] = cleaned_response
                row_data[source_key] = str(response_info[response_key][1]) if response_info[response_key][1] else ''
            writer.writerow(row_data)

# Appeler la fonction avec vos données
write_to_csv(dict_output, output_csv_path)

# Afficher un message de confirmation
print(f"Les données ont été écrites dans le fichier CSV : {output_csv_path}")

Les données ont été écrites dans le fichier CSV : output_llama_extract.csv


# Extraction de metadata spécifique à notre expérimentation

### import

### Extraction métadata (titre et sous titre) pour chaque document 

In [19]:
# Spécifiez le chemin du dossier que vous souhaitez lister
dossier_path = "/share/data/fiches_seismes"

# Utilisez glob pour obtenir la liste des fichiers PDF dans le dossier
liste_documents_pdf = glob.glob(os.path.join(dossier_path, "*.pdf"))

# Imprimez la liste des documents PDF
print("Liste des documents PDF dans le dossier:")
#for document_pdf in liste_documents_pdf:
#    print(document_pdf)

# Éléments à exclure
elements_a_exclure = ["/share/data/fiches_seismes/IRSN-seisme-Layon-21062019_V1.pdf","/share/data/fiches_seismes/IRSN-Seisme-St-Paul-Ubaye-07042014.pdf",
                     "/share/data/fiches_seismes/irsn_Seisme-Chine_052008.pdf","/share/data/fiches_seismes/IRSN_Fiche-seisme-PETRINJA-2020_vf.pdf",
                     "/share/data/fiches_seismes/IRSN-seisme_Okhotsk_240513.pdf","/share/data/fiches_seismes/irsn_seisme_notohanto_japon_042007.pdf",
                     "/share/data/fiches_seismes/IRSN_Fiche-seisme-Turquie-20230207.pdf","/share/data/fiches_seismes/IRSN_seisme-Chili-V3_07042010.pdf",
                     "/share/data/fiches_seismes/IRSN-Seismes-Ridgecrest-Californie-201907.pdf","/share/data/fiches_seismes/irsn_Seisme-Italie_042009.pdf"]

# Créez une nouvelle liste en excluant les éléments spécifiés
doc_fonctionnels = list(filter(lambda x: x not in elements_a_exclure, liste_documents_pdf))

# Affichez la liste mise à jour
print(doc_fonctionnels)
doc_all = []
for document_pdf in doc_fonctionnels:
    print(document_pdf)
    loader = PDFMinerPDFasHTMLLoader(document_pdf)
    data = loader.load()[0]   # entire PDF is loaded as a single Document
    
    soup = BeautifulSoup(data.page_content,'html.parser')
    content = soup.find_all('div')
    
    cur_fs = None
    cur_text = ''
    snippets = []   # first collect all snippets that have the same font size
    for c in content:
        sp = c.find('span')
        if not sp:
            continue
        st = sp.get('style')
        if not st:
            continue
        fs = re.findall('font-size:(\d+)px',st)
        if not fs:
            continue
        fs = int(fs[0])
        if not cur_fs:
            cur_fs = fs
        if fs == cur_fs:
            cur_text += c.text
        else:
            snippets.append((cur_text,cur_fs))
            cur_fs = fs
            cur_text = c.text
    snippets.append((cur_text,cur_fs))
    header = []
    compt = 0
    for s in snippets:
        if compt < 100 and s[1] >= 11 : 
            header.append(s)
            compt+=len(s[0])
    header.append(data.metadata['source'])
    doc_all.append(header)


Liste des documents PDF dans le dossier:
['/share/data/fiches_seismes/irsn_seisme_argeles_gazost_112006.pdf', '/share/data/fiches_seismes/irsn_seisme_chinon_11_2006.pdf', '/share/data/fiches_seismes/IRSN_seisme-haiti_130110.pdf', '/share/data/fiches_seismes/IRSN-Seisme-Civaux-09092013.pdf', '/share/data/fiches_seismes/IRSN_seisme_turquie-23102011.pdf', '/share/data/fiches_seismes/irsn_seisme_niigata_japon_092007.pdf', '/share/data/fiches_seismes/irsn_seisme_martinique_122007.pdf', '/share/data/fiches_seismes/IRSN_seisme_france-26022012_V2.pdf', '/share/data/fiches_seismes/Note-Information-Seisme-Teil-14112019.pdf', '/share/data/fiches_seismes/IRSN_seisme_Italie-20052012.pdf', '/share/data/fiches_seismes/IRSN_seisme-Japon_V2-22042011.pdf', '/share/data/fiches_seismes/IRSN_Seisme_Lorca-Espagne_18052011.pdf', '/share/data/fiches_seismes/irsn_Seisme-Mexique_012003.pdf', '/share/data/fiches_seismes/IRSN-Seisme-Norcia-Italie-301016_V1.pdf', '/share/data/fiches_seismes/irsn_Seisme-Afghanistan

#### Filtre des métadata intéressantees

In [20]:
liste_filtrée = []
for doc in doc_all:
    # Filtrer les deux premiers éléments en fonction de leur valeur
    liste_filtrée.append([item for item in doc if not (item[0].strip() == 'I  N  F  O  R  M  A  T  I  O  N' or re.search(r'\bFS\b', item[0]))])
    
for titre in liste_filtrée :
    print(titre)

[('Séisme d’Argelès-Gazost\n', 22), ('Vendredi 17 novembre 2006\nMagnitude = 5.2 à 18h19 (TU)\n', 13), '/share/data/fiches_seismes/irsn_seisme_argeles_gazost_112006.pdf']
[('Séisme de Chinon\n', 22), ('Dimanche 5 novembre 2006\nMagnitude = 4.1 à 0h37 (TU)\nRappel des faits\n', 13), '/share/data/fiches_seismes/irsn_seisme_chinon_11_2006.pdf']
[('Séisme d’Haïti \n(Caraïbes)\n', 21), ('du mardi 12 janvier 2010\nMagnitude = 7,0 à 21h53 (TU)\n', 14), '/share/data/fiches_seismes/IRSN_seisme-haiti_130110.pdf']
[('Séisme à Lhommaizé \n(Vienne)  \n', 21), ('Du lundi 9 septembre 2013 (20h50 TU)\nMagnitude = 3,1 (Ml) \n', 14), '/share/data/fiches_seismes/IRSN-Seisme-Civaux-09092013.pdf']
[('Séisme de Van (Turquie) \n', 22), ('du dimanche 23 octobre 2011 (10h41 TU)\nMagnitude = 7,2\n', 13), '/share/data/fiches_seismes/IRSN_seisme_turquie-23102011.pdf']
[('Séisme de Chu-Etsu-Oki (Japon)\n', 22), ('du lundi 16 juillet 2007\nMagnitude = 6,6 à 1h13 (TU)\n', 13), '/share/data/fiches_seismes/irsn_seisme

#### Ingestion des métadatas dans les nodes

In [22]:
print(doc)

[('I  N  F  O  R  M  A  T  I  O  N \n', 12), ('Séisme à La Rochelle \n(Nord du bassin aquitain)  \n', 22), ('Du 28 avril 2016 (6h46 TU)\nMagnitude = 5,2 (Ml - CEA) \n', 14), '/share/data/fiches_seismes/IRSN-Seisme-LaRochelle-28042016.pdf']


In [25]:
for node in doc_load :
    for titre in liste_filtrée :
        if titre[-1] == node.metadata['file_path']:
            node.metadata ['document_title'] = titre[:-1]
for node in doc_load :
    print(node.metadata)

{'page_label': '1', 'file_name': 'IRSN Fiche seisme MULHOUSE 13092022.pdf', 'file_path': '/share/data/fiches_seismes/IRSN Fiche seisme MULHOUSE 13092022.pdf', 'file_type': 'application/pdf', 'file_size': 589122, 'creation_date': '2023-12-04', 'last_modified_date': '2023-11-30', 'last_accessed_date': '2024-03-06', 'document_title': [('INFORMATION : Séisme de Mulhouse (Haut-Rhin) \n', 24), ('Séisme de Mulhouse (Haut-Rhin) \ndu 10 septembre 2022 (15h58 TU – 17h58 locale)  \nMagnitude = 3,9 (Mw –Géoazur)\n Rappel des faits. \n', 11)]}
{'page_label': '2', 'file_name': 'IRSN Fiche seisme MULHOUSE 13092022.pdf', 'file_path': '/share/data/fiches_seismes/IRSN Fiche seisme MULHOUSE 13092022.pdf', 'file_type': 'application/pdf', 'file_size': 589122, 'creation_date': '2023-12-04', 'last_modified_date': '2023-11-30', 'last_accessed_date': '2024-03-06', 'document_title': [('INFORMATION : Séisme de Mulhouse (Haut-Rhin) \n', 24), ('Séisme de Mulhouse (Haut-Rhin) \ndu 10 septembre 2022 (15h58 TU – 17h5

### Création du vectore store à partir des nodes avec les bonnes métadata

In [26]:
#query_engine = index.as_query_engine(response_mode="compact")
vector_index_date = VectorStoreIndex(doc_load, service_context=service_context, show_progress = True)

Generating embeddings: 100%|██████████| 112/112 [00:00<00:00, 171.63it/s]


### Sauvegarde embedding

In [27]:
vector_index_date.storage_context.persist(persist_dir="/share/LLM_models/Embedding_store_MN")


### Chargement embedding

In [17]:
from llama_index import StorageContext, load_index_from_storage
# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir="/share/LLM_models/Embedding_store_T_QA_MN_S")

# load index
index = load_index_from_storage(storage_context,service_context=service_context)

### Création de l'outil de question 

In [18]:
query_engine = vector_index_date.as_query_engine(response_mode="compact")

In [19]:
reponse = query_engine.query(" Quelle est la magnitude, la localisation et les conséquences régionales du séisme de Mulhouse ?")
print(reponse)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


IRSN Fiche seisme MULHOUSE 13092022.pdf
Note-Information-Seisme-Teil-14112019.pdf

The magnitude of the earthquake in Mulhouse is 3.9 (Mw - Géoazur). The localization of the earthquake is 7.471° E and 47.669 °N. The consequences of the earthquake were felt throughout the region of Grand Est. At 13 September 2022, 2068 témoignages were collected by the BCSF, allowing the establishment of a map of seismic intensities reaching a maximum of IV, corresponding to a perception of moderate intensity and causing no damage to buildings.


### Process de sauvegarde des réponses dans un csv

In [21]:
mon_dict = {
    'IRSN Fiche seisme MULHOUSE 13092022.pdf': ['Quelle est la magnitude, la localisation et les conséquences régionales du séisme de Mulhouse ?', "Comment le séisme de Mulhouse est-il lié à la tectonique du Fossé Rhénan, et quelles sont les prévisions quant à l'activité sismique future dans la région ?"],
    'IRSN_Fiche-Seisme-Maroc.pdf': ['Quelles sont les caractéristiques principales du séisme au Maroc du 8 septembre 2023, telles que sa magnitude, sa localisation, et quelles ont été ses conséquences régionales ?', "Comment s'inscrit le séisme du 8 septembre 2023 dans le contexte sismotectonique du Maroc, et quel impact a-t-il eu sur les installations nucléaires dans la région ?"],
    'IRSN_Fiche-Seisme-Ouest-France-16-juin-2023_V1-17062023.pdf': ["Quelles sont les caractéristiques principales du séisme dans l'ouest de la France du 16 juin 2023, notamment sa localisation, sa magnitude, et quels sont les impacts rapportés dans la région épicentrale ?", "Comment se situe le séisme du 16 juin 2023 par rapport à l'activité sismique historique de la région, et quel a été l'impact sur les installations nucléaires à proximité, telles que la centrale nucléaire de Civaux ?"],
    'irsn_seisme_argeles_gazost_112006.pdf': ["Pouvez-vous fournir des informations sur le séisme d'Argelès-Gazost du 17 novembre 2006, notamment sa magnitude, son épicentre, et ses impacts locaux ?", "Quel a été l'impact du séisme d'Argelès-Gazost sur les installations nucléaires de la région ?"],
    'irsn_seisme_chinon_11_2006.pdf': ['Pouvez-vous fournir des informations sur le séisme de Chinon du 5 novembre 2006, notamment sa magnitude, son épicentre, et ses impacts locaux ?', "Quel a été l'impact du séisme de Chinon sur les installations nucléaires de la région ?"],
    'IRSN_seisme_Christchurch-NZ_23022011.pdf': ['Quels sont les impacts du séisme de Christchurch du 21 février 2011 par rapport à celui de septembre 2010 en Nouvelle-Zélande?', 'Quelle est la situation des installations nucléaires en Nouvelle-Zélande et quelles sont les caractéristiques de la faille de GreenDale?'],
    'IRSN_seisme_france-26022012_V2.pdf': ['Quels sont les détails concernant le séisme à Saint-Paul-sur-Ubaye du 26 février 2012?', "Comment la sismicité historique et actuelle dans la vallée de l'Ubaye est-elle liée au séisme du 26 février 2012?"],
    'IRSN_seisme_Italie-20052012.pdf': ["Quel est le contexte sismotectonique de la région touchée par le séisme de l'Emilia-Romagna?", "Quelle est l'estimation de l'aléa sismique et du mouvement sismique en basse plaine du Pô?"],
    'IRSN_Seisme_Lorca-Espagne_18052011.pdf': ['Quels sont les détails concernant le séisme de Lorca en Espagne du 11 mai 2011?', 'Quel est le contexte sismique de la France autour du 11 mai 2011?'],
    'irsn_seisme_martinique_122007.pdf': ['Quand a eu lieu le séisme de Martinique et quelle était sa magnitude?', "Où était situé l'épicentre du séisme de Martinique?"],
    'irsn_seisme_niigata_japon_092007.pdf': ['Quelles étaient les conséquences humaines du séisme de Chu-Etsu-Oki?','Comment le Japon se situe-t-il du point de vue sismotectonique par rapport à la convergence des plaques tectoniques?'],
    'IRSN_seisme_sumatra-11042012.pdf': ['Quand ont eu lieu les séismes de Sumatra en Indonésie, et quelle était leur magnitude?', "Quelle était la distance entre la centrale nucléaire la plus proche (Kalpakkam) et l'épicentre du séisme de Sumatra, et quels étaient les effets attendus sur la centrale?"],
    'IRSN_seisme_turquie-23102011.pdf': ['Quels ont été les effets du séisme de Van en termes de dégâts matériels et humains?', 'Où était située la centrale nucléaire la plus proche du séisme de Van, et quelles étaient les conséquences sur cette centrale?'],
    'IRSN_seisme_virginie-US-23082011.pdf': ["Dans quelle région s'est produit le séisme de Virginie, et qu'est-ce que la « Central Virginia Seismic Zone » ?", " Quel est le contexte sismotectonique de l'Est des États-Unis, et pourquoi la sismicité n'y est-elle pas négligeable?"],
    'irsn_Seisme-Afghanistan_032002.pdf': ['Quand a eu lieu le séisme en Afghanistan, et quelle était sa magnitude?', 'Quelles sont les différentes estimations de la magnitude du séisme en Afghanistan?'],
    'irsn_seisme-algerie_052003.pdf': ['Quels sont les effets de site probables dans les localités de Rouïba et Boumerdès?',"Quel est le contexte tectonique de l'Algérie et comment cela contribue-t-il à la sismicité de la région?"],
    'irsn_Seisme-Allemagne_122004.pdf': ['Quelle est la proximité du séisme de Fribourg par rapport à la centrale nucléaire de Fessenheim?', "Quelle est la distance entre l'épicentre du séisme de Fribourg et les centrales nucléaires suisses de Leipstadt et Beznau?"],
    'irsn_Seisme-Besancon_022004.pdf': ['Quand a eu lieu le séisme de Besançon et quelle était sa magnitude?'],
    'IRSN_seisme-haiti_130110.pdf': ["Où était situé l'épicentre du séisme d'Haïti, et quels ont été les effets notables?", "Quel est le contexte sismotectonique de la région où s'est produit le séisme d'Haïti?"],
    'irsn_Seisme-Inde_012001.pdf': ['Quels ont été les effets notables du séisme de Gurajat, et quel était le mécanisme de la faille en jeu?', ' Pourquoi le séisme de Gurajat a-t-il été considéré comme une surprise relative?'],
    'irsn_Seisme-Japon_052003.pdf': ["Quand et où s'est produit le séisme de Miyagi au Japon, et quelle était sa magnitude?", 'Quels sont les impacts du séisme de Miyagi en termes de dégâts matériels, de victimes et de répliques?'],
    'IRSN_Seisme-Japon_Replique-7-avril_V1-08042011.pdf': ['Quelles sont les actions entreprises par les autorités japonaises pour évaluer les effets de cette réplique?',"Comment la centrale nucléaire d'Onagawa a-t-elle été affectée par cette réplique en termes d'accélérations du sol?"],
    'IRSN_seisme-Japon_V2-22042011.pdf': ["Quand et où s'est produit le séisme de Tohoku au Japon du 11 mars 2011, et quelle était sa magnitude?", "Quelle était la localisation de l'épicentre et quels sont les détails sur la magnitude du séisme de Tohoku au Japon du 11 mars 2011?"],
    'IRSN_seisme-Manosque_V1-12072010.pdf': ['Quelle est la magnitude du séisme survenu à Manosque le 8 juillet 2010, et quelle est la version du document que vous consultez ?', 'Comment la population a-t-elle réagi à la secousse sismique, et quelles sont les zones les plus fortement touchées pour le séisme du 8 juillet 2010 ? ?'],
    'irsn_Seisme-Mexique_012003.pdf': ["Quels sont les dégâts rapportés suite au séisme, et quelle est la particularité de la ville de Mexico en termes d'effet sismique ?", 'Quelle est la situation des installations nucléaires au Mexique, et comment le séisme du 21 janvier 2003 a-t-il affecté la centrale nucléaire de Laguna Verde '],
    'IRSN-Seisme-Amatrice-Italie-240816_V1.pdf': ["Quelle est la magnitude du séisme d'Amatrice qui s'est produit le 24 août 2016, et où était situé son épicentre ?", 'Quels ont été les effets du séisme  du 24 août 2016 en termes de victimes, de blessés et de dégâts matériels ?'],
    'IRSN-Seisme-Charente-Maritime-France-20190320_v1.1.pdf': ['Quels ont été les effets du séisme de Montendre en termes de ressenti par la population et et de dégâts matériels ?', 'Comment le séisme de Montendre de 2019 se compare-t-il aux plus forts séismes historiques de la région ?'],
    'IRSN-Seisme-Civaux-09092013.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Lhommaizé le 9 septembre 2013, et où était situé son épicentre ?", 'Quels ont été les effets du séisme de Lhommaizé en termes de ressenti par la population et de dégâts matériels ?'],
    'IRSN-Seisme-Kent-22052015.pdf': ["Quels ont été les impacts du séisme dans le Kent le 22 mai 2015 en termes de secousses ressenties, d'intensités et de mécanisme sismique associé ?", 'Quelle est la sismicité historique dans le détroit du pas de Calais, et quelles sont les recherches menées pour mieux comprendre le risque sismique dans cette région ?'],
    'IRSN-Seisme-Elazig-Turquie-24012020.pdf': ["Quelle est la géodynamique de la région et la sismicité régionale autour d'Elazig ?", 'Quelles sont les conséquences sur les installations nucléaires à proximité, en particulier la centrale nucléaire de Metsamor en Arménie, et quels sont les projets de centrales nucléaires en Turquie ?'],
    'IRSN-Seisme-Fukushima-13022021.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Fukushima le 13 février 2021, et où était situé son épicentre ?", 'Quels ont été les impacts du séisme de Fukushima en termes de victimes, de dommages matériels et de secousses ressenties dans la région ?'],
    'IRSN-Seisme-LaRochelle-28042016.pdf': ["Quelle est la magnitude du séisme qui s'est produit à La Rochelle le 28 avril 2016, et où était situé son épicentre ?", "Quelle est la sismicité historique et actuelle au nord de l'Aquitaine, et comment le séisme de 2016 s'inscrit-il dans ce contexte ?"],
    'IRSN-Seisme-Manosque-19092012.pdf': ["Quels ont été les faits marquants du séisme à Villeneuve le 19 septembre 2012, en termes de répliques, de secousses ressenties, et d'historique sismique de la région ?", "Quelle est la géométrie de la Faille de la Moyenne Durance, et comment le séisme de 2012 s'inscrit-il dans l'activité de cette faille ?"],
    'IRSN-Seisme-Napa-24082014.pdf': ["Quelle est la magnitude du séisme qui s'est produit à Napa le 24 août 2014, et quelles sont les principales caractéristiques de ce séisme ?", 'Quelle est la localisation du séisme de Napa par rapport aux failles tectoniques de la région, et quelles sont les caractéristiques tectoniques de cette zone ?'],
    'IRSN-Seisme-Norcia-Italie-301016_V1.pdf': ["Quelle est la magnitude du séisme qui s'est produit en Italie centrale le 30 octobre 2016, et quelles sont les principales caractéristiques de ce séisme ?", "Quels sont les enseignements tirés de la séquence sismique en Italie centrale, et en quoi cette expérience peut-elle être importante pour la compréhension des risques sismiques dans d'autres régions, y compris en France ?"],
    'IRSN-Seisme-Vendee-France-20180212.pdf': ['Quand et où a eu lieu le séisme de Saint-Hilaire-de-Voust en 2018, quelle était sa magnitude, et quels sont les dégâts signalés?', "Quel a été l'impact du séisme de Saint-Hilaire-de-Voust de 2018 sur les installations nucléaires, et quelles sont les installations nucléaires les plus proches de l'épicentre?"]}

In [22]:
reponses_dict = {}  
for cle, questions in mon_dict.items():
    print(f'Clé: {cle}')
    
    # Liste pour stocker les réponses aux questions
    reponses = []
    # Itérer sur chaque question pour la clé
    for i, question in enumerate(questions, start=1):
        query = question
        reponses.append(query_engine.query(query))  
    # Stocker les réponses dans le dictionnaire de réponses
    reponses_dict[cle] = reponses


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN Fiche seisme MULHOUSE 13092022.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN_Fiche-Seisme-Maroc.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN-Seisme-Norcia-Italie-301016_V1.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Clé: IRSN-Seisme-Vendee-France-20180212.pdf


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [23]:
dict_output = {}

for cle in reponses_dict:
    if cle == 'irsn_Seisme-Besancon_022004.pdf':
        continue

    # Initialiser une liste pour stocker les réponses
    reponses_list = []

    # Itérer sur chaque question pour la clé
    for i in range(2):
        reponses_list.append({
            f'Réponse {i}': [reponses_dict[cle][i].response, reponses_dict[cle][i].metadata[list(reponses_dict[cle][i].metadata.keys())[0]]['file_name'] + ' & ' + reponses_dict[cle][i].metadata[list(reponses_dict[cle][i].metadata.keys())[1]]['file_name']]
        })

    # Ajouter la liste des réponses au dictionnaire de sortie
    dict_output[cle] = reponses_list

# Afficher le résultat pour une clé spécifique
print(dict_output['IRSN Fiche seisme MULHOUSE 13092022.pdf'])


[{'Réponse 0': ['\nThe magnitude of the earthquake in Mulhouse is 3.9 (Mw - Géoazur) and its localization is 7.471° E and 47.669 °N. The consequences of the earthquake were felt throughout the region of Grand Est. At 13 September 2022, 2068 témoignages were collected by the BCSF, allowing for the establishment of a map of seismic intensities that reached a maximum of IV, corresponding to a perception of moderate intensity and no damage to buildings.', 'IRSN Fiche seisme MULHOUSE 13092022.pdf & Note-Information-Seisme-Teil-14112019.pdf']}, {'Réponse 1': ["\nThe given context information does not provide any direct information about the link between the Mulhouse earthquake and the tectonics of the Rhine Fossa. However, it does mention that the earthquake occurred at 14 km southeast of Mulhouse and 15 km north of Basel, with a localization according to the RENASS (Réseau national d'études sismiques). The RENASS is a French network that studies seismic activity and its potential impact on 

In [24]:
import csv
output_csv_path = 'output_llama_extract_MN.csv'
# Fonction pour écrire les données dans le fichier CSV
def write_to_csv(data, csv_path):
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        # Définir les colonnes du CSV
        fieldnames = ['Nom du document', 'Réponse 0', 'Source 0', 'Réponse 1', 'Source 1']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';')  # Ajout du délimiteur

        # Écrire l'en-tête
        writer.writeheader()

        # Écrire les données
        for document, responses in data.items():
            row_data = {'Nom du document': document}
            for i, response_info in enumerate(responses):
                response_key = f'Réponse {i}'
                source_key = f'Source {i}'
                
                # Enlever le premier caractère (retour chariot) du string réponse
                cleaned_response = response_info[response_key][0][1:] if response_info[response_key][0] else ''
                
                row_data[response_key] = cleaned_response
                row_data[source_key] = str(response_info[response_key][1]) if response_info[response_key][1] else ''
            writer.writerow(row_data)

# Appeler la fonction avec vos données
write_to_csv(dict_output, output_csv_path)

# Afficher un message de confirmation
print(f"Les données ont été écrites dans le fichier CSV : {output_csv_path}")

Les données ont été écrites dans le fichier CSV : output_llama_extract_MN.csv


# Process Création et sauvegarde json

In [7]:
# Spécifiez le chemin du dossier que vous souhaitez lister
dossier_path = "/share/data/fiches_seismes"

# Utilisez glob pour obtenir la liste des fichiers PDF dans le dossier
liste_documents_pdf = glob.glob(os.path.join(dossier_path, "*.pdf"))

# Imprimez la liste des documents PDF
print("Liste des documents PDF dans le dossier:")
#for document_pdf in liste_documents_pdf:
#    print(document_pdf)

# Éléments à exclure
elements_a_exclure = ["/share/data/fiches_seismes/IRSN-seisme-Layon-21062019_V1.pdf","/share/data/fiches_seismes/IRSN-Seisme-St-Paul-Ubaye-07042014.pdf",
                     "/share/data/fiches_seismes/irsn_Seisme-Chine_052008.pdf","/share/data/fiches_seismes/IRSN_Fiche-seisme-PETRINJA-2020_vf.pdf",
                     "/share/data/fiches_seismes/IRSN-seisme_Okhotsk_240513.pdf","/share/data/fiches_seismes/irsn_seisme_notohanto_japon_042007.pdf",
                     "/share/data/fiches_seismes/IRSN_Fiche-seisme-Turquie-20230207.pdf","/share/data/fiches_seismes/IRSN_seisme-Chili-V3_07042010.pdf",
                     "/share/data/fiches_seismes/IRSN-Seismes-Ridgecrest-Californie-201907.pdf","/share/data/fiches_seismes/irsn_Seisme-Italie_042009.pdf"]

# Créez une nouvelle liste en excluant les éléments spécifiés
doc_fonctionnels = list(filter(lambda x: x not in elements_a_exclure, liste_documents_pdf))

# Affichez la liste mise à jour
print(doc_fonctionnels)


Liste des documents PDF dans le dossier:
['/share/data/fiches_seismes/irsn_seisme_argeles_gazost_112006.pdf', '/share/data/fiches_seismes/irsn_seisme_chinon_11_2006.pdf', '/share/data/fiches_seismes/IRSN_seisme-haiti_130110.pdf', '/share/data/fiches_seismes/IRSN-Seisme-Civaux-09092013.pdf', '/share/data/fiches_seismes/IRSN_seisme_turquie-23102011.pdf', '/share/data/fiches_seismes/irsn_seisme_niigata_japon_092007.pdf', '/share/data/fiches_seismes/irsn_seisme_martinique_122007.pdf', '/share/data/fiches_seismes/IRSN_seisme_france-26022012_V2.pdf', '/share/data/fiches_seismes/Note-Information-Seisme-Teil-14112019.pdf', '/share/data/fiches_seismes/IRSN_seisme_Italie-20052012.pdf', '/share/data/fiches_seismes/IRSN_seisme-Japon_V2-22042011.pdf', '/share/data/fiches_seismes/IRSN_Seisme_Lorca-Espagne_18052011.pdf', '/share/data/fiches_seismes/irsn_Seisme-Mexique_012003.pdf', '/share/data/fiches_seismes/IRSN-Seisme-Norcia-Italie-301016_V1.pdf', '/share/data/fiches_seismes/irsn_Seisme-Afghanistan

In [8]:
for doc in doc_fonctionnels :
    try:
        # chargement du bon document
        reader_json=  SimpleDirectoryReader(input_files = [doc], required_exts=[".pdf"])
        doc_json = reader_json.load_data()
        # création de 'lindex pour ce document
        vector_index_json = VectorStoreIndex(doc_json, service_context=service_context, show_progress = True)
        query_engine = vector_index_json.as_query_engine(response_mode="compact",similarity_top_k=len(doc_json))
    
        # prompt pour remplir le json
        reponse = query_engine.query(""" Rempli ce json :
            {
                "seisme": {
                  "lieu": "non spécifié",
                  "date": "non spécifié",
                  "heure_utc": "non spécifié",
                  "heure_locale": "non spécifié",
                  "magnitude": "non spécifié",
                  "localisation": {
                    "latitude": "non spécifiée",
                    "longitude": "non spécifiée"
                  },
                  "repliques": {
                    "nombre": "non spécifié",
                    "magnitudes": {
                      "min": "non spécifié",
                      "max": "non spécifié"
                    }
                  },
                  "impact": {
                    "humain": "non spécifié",
                    "matériel": "non spécifié"
                  },
                  "contexte_sismotectonique": {
                    "description": "non spécifié"
                  },
                  "sismicite_historique": {
                    "evenements_precedents": [
                      {
                        "date": "non spécifiée",
                        "magnitude": "non spécifiée",
                        "localisation": "non spécifiée",
                        "victimes_estimees": "non spécifié"
                      }
                    ]
                  },
                  "impact_installations_nucleaires": {
                    "reacteur_recherche": {
                      "puissance": "non spécifié",
                      "localisation": "non spécifiée",
                      "impact": "non spécifié"
                    }
                  },
                  "references": [
                    "non spécifié"
                  ]
                }
                }
                
           """)
        # formattage du string en json
        data = json.loads(reponse.response.replace("\\", "") )
        # Chemin vers le fichier JSON de sortie
        doc_name = reponse.metadata[list(reponse.metadata.keys())[0]]['file_name'].replace(".pdf", "")
        
        output_file_path = f"json_v2/{doc_name}.json"
        
        # Écriture des données dans le fichier JSON
        with open(output_file_path, 'w') as json_file:
            json.dump(data, json_file, indent=4, ensure_ascii=False)
        
        print("Output écrit avec succès dans le fichier:", output_file_path)
    except Exception as e:
        print(f"Erreur lors du traitement du document {doc}: {str(e)}")
        doc_name = reponse.metadata[list(reponse.metadata.keys())[0]]['file_name'].replace(".pdf", "")
        # En cas d'erreur, écrire la réponse brute dans un fichier texte
        error_output_file_path = f"txt_v2/{doc_name}_error.txt"
        with open(error_output_file_path, 'w') as error_file:
            error_file.write(reponse.response)
        print(f"Réponse brute écrite dans le fichier texte: {error_output_file_path}")
        continue  # Passer au document suivant en cas d'erreur

Generating embeddings: 100%|██████████| 3/3 [00:00<00:00,  4.57it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output écrit avec succès dans le fichier: json_v2/irsn_seisme_argeles_gazost_112006.json


Generating embeddings: 100%|██████████| 3/3 [00:00<00:00, 136.74it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Erreur lors du traitement du document /share/data/fiches_seismes/irsn_seisme_chinon_11_2006.pdf: Extra data: line 50 column 1 (char 1349)
Réponse brute écrite dans le fichier texte: txt_v2/irsn_seisme_chinon_11_2006_error.txt


Generating embeddings: 100%|██████████| 2/2 [00:00<00:00, 90.96it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Erreur lors du traitement du document /share/data/fiches_seismes/IRSN_seisme_sumatra-11042012.pdf: Expecting ',' delimiter: line 52 column 2 (char 2222)
Réponse brute écrite dans le fichier texte: txt_v2/IRSN_seisme_sumatra-11042012_error.txt


Generating embeddings: 100%|██████████| 2/2 [00:00<00:00, 108.91it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Output écrit avec succès dans le fichier: json_v2/IRSN-Seisme-Kent-22052015.json


Generating embeddings: 100%|██████████| 2/2 [00:00<00:00, 128.53it/s]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Erreur lors du traitement du document /share/data/fiches_seismes/IRSN-Seisme-LaRochelle-28042016.pdf: Expecting ',' delimiter: line 51 column 2 (char 1418)
Réponse brute écrite dans le fichier texte: txt_v2/IRSN-Seisme-LaRochelle-28042016_error.txt


In [1]:
pip freeze

absl-py==2.0.0
accelerate==0.25.0
aiohttp==3.8.6
aiosignal==1.3.1
annotated-types==0.6.0
anyio==3.7.1
asttokens==2.4.0
astunparse==1.6.3
async-timeout==4.0.3
attrs==23.1.0
backcall==0.2.0
backoff==2.2.1
bcrypt==4.0.1
beautifulsoup4==4.12.3
bitsandbytes==0.41.3
bs4==0.0.2
cachetools==5.3.2
certifi==2023.7.22
cffi==1.16.0
charset-normalizer==3.3.1
chroma-hnswlib==0.7.3
chromadb==0.4.17
click==8.1.7
coloredlogs==15.0.1
comm==0.1.4
cryptography==42.0.1
dataclasses-json==0.6.2
datasets==2.17.0
debugpy==1.8.0
decorator==5.1.1
Deprecated==1.2.14
dill==0.3.8
dirtyjson==1.0.8
distro==1.9.0
dm-tree==0.1.8
evaluate==0.4.1
executing==2.0.0
faiss-cpu==1.7.4
fastapi==0.104.1
filelock==3.12.4
flatbuffers==23.5.26
frozenlist==1.4.0
fsspec==2023.10.0
gast==0.5.4
google-auth==2.23.3
google-auth-oauthlib==1.0.0
google-pasta==0.2.0
googleapis-common-protos==1.61.0
gpt4all==2.0.2
greenlet==3.0.1
grpcio==1.59.0
h11==0.14.0
h5py==3.10.0
httpcore==1.0.2
httptools==0.6.1
httpx==0.26.0
huggingface-hub==0.20.3
h